# 1- Tabelas Auxiliares  

Serão utilizadas para interpretar os códigos da base de dados e, por isso, devem ser tratadas no primeiro passo.

### Distribuição Territorial Brasileira - IBGE  

Devido a uma demonstração de interesse por parte de uma empresa importadora de fertilizantes sediada em Pouso Alegre/MG, esse projeto visa prever as importações deste município e de todos os demais na mesma região geográfica.  

O arquivo coletado e armazenado no endereço a seguir será descompactado e filtrado, deixando passar somente as informações referentes a região de Pouso Alegre/MG.

* '~/git/BD/IBGE/DTB_2022.zip'

Ao procurar o nome "Pouso Alegre" dentre os tipos de sub-divisão geograficas, se confirma que este município é classificado como centro de uma "Região Geográfica **Intermediária**" do estado de Minas Gerais. As regiões deste tipo contém nelas uma segunda sub-divisão, denominada "Região Geográfica **Imediata**" onde, por fim, se encontra a última e menor subdivisão, denominada Município.

Ao final, o arquivo será salvo em :

* 'git/Importacao-por-SH-e-Municipio/temp/i_f_r_pa.csv'

In [41]:
%run '~/git/Bibliotecas.ipynb'

In [1]:
import zipfile
import io
import os
import pandas as pd

In [2]:
caminho_arquivo_zip = '~/git/BD/IBGE/DTB_2022.zip'
string_procurada = 'MUNICIPIO'

# Expandir o caminho do arquivo
caminho_arquivo_zip = os.path.expanduser(caminho_arquivo_zip)

# Variável para armazenar o conteúdo do arquivo XLS
m = None

# Descompactar o arquivo em memória
with zipfile.ZipFile(caminho_arquivo_zip, 'r') as zip_ref:
    # Obter uma lista de nomes de arquivos no zip
    nomes_arquivos = zip_ref.namelist()

    for nome_arquivo in nomes_arquivos:
        # Verificar se a string 'MUNICIPIO' está presente no nome do arquivo e se o arquivo é um XLS
        if string_procurada.upper() in nome_arquivo.upper() and nome_arquivo.lower().endswith('.xls'):
            # Ler o conteúdo do arquivo no zip
            conteudo_arquivo = zip_ref.read(nome_arquivo)

            # Utilizar o pandas para ler o arquivo XLS
            m = pd.read_excel(io.BytesIO(conteudo_arquivo), skiprows=6)

            # Exibir as primeiras linhas do DataFrame (opcional)
            display(m.head())

            # Aqui você pode fazer operações adicionais com o conteúdo do arquivo, se necessário

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'


,UF,Nome_UF,Região Geográfica Intermediária,Nome Região Geográfica Intermediária,Região Geográfica Imediata,Nome Região Geográfica Imediata,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,Código Município Completo,Nome_Município
0,11,Rondônia,1102,Ji-Paraná,110005,Cacoal,2,Leste Rondoniense,6,Cacoal,15,1100015,Alta Floresta D'Oeste
1,11,Rondônia,1102,Ji-Paraná,110005,Cacoal,2,Leste Rondoniense,6,Cacoal,379,1100379,Alto Alegre dos Parecis
2,11,Rondônia,1101,Porto Velho,110002,Ariquemes,2,Leste Rondoniense,3,Ariquemes,403,1100403,Alto Paraíso
3,11,Rondônia,1102,Ji-Paraná,110004,Ji-Paraná,2,Leste Rondoniense,5,Alvorada D'Oeste,346,1100346,Alvorada D'Oeste
4,11,Rondônia,1101,Porto Velho,110002,Ariquemes,2,Leste Rondoniense,3,Ariquemes,23,1100023,Ariquemes


In [3]:
import os

folder_path = '/home/andre301267/git/Importacao-por-SH-e-Municipio/temp'

# Check if the folder already exists
if not os.path.exists(folder_path):
    # Create the folder
    os.makedirs(folder_path)
    print(f"Folder '{folder_path}' created successfully.")
else:
    print(f"Folder '{folder_path}' already exists.")


Folder '/home/andre301267/git/Importacao-por-SH-e-Municipio/temp' created successfully.


In [4]:
# Filtra deixando passar somente municíppios da região de Pouso Alegere
m_r_pa=m[
    (m['Nome Região Geográfica Intermediária'].astype(str).str.contains('Pouso Alegre'))
    &
    (m['Nome_UF']=='Minas Gerais')
]

# seleciona somente a coluna com o nome dos municípios
m_r_pa=m_r_pa[m_r_pa.columns[[12]].tolist()]
# renomeia a coluna em busca de uniformidade com outros arquivos 
m_r_pa=m_r_pa.rename(columns={'Nome_Município':'Município'}).reset_index(drop=True)

# Salva 
m_r_pa.to_csv('~/git/comex_sh_mun/temp/m_r_pa.csv',index=False)

# Imprime
print('\nMunicípios da Região de Pouso Alegre:')
display(m_r_pa)
print('\nFonte: IBGE')


Municípios da Região de Pouso Alegre:


,Município
0,Aiuruoca
1,Alagoa
2,Albertina
3,Andradas
4,Baependi
...,...
75,Tocos do Moji
76,Toledo
77,Turvolândia
78,Virgínia



Fonte: IBGE


# 2- Importação de Fertilizantes por Município  

No capítulo anterior, a base de dados com importações por Município, denominada IMP_COMPLETA_MUN.zip, e as tabelas complementares, foram baixadas e salvas localmente na pasta:  

* '~/git/BD/Comexstat/'

Descompacta o arquivo e salva localmente em pasta específica

In [5]:
import zipfile
import os

caminho_arquivo_zip = '/home/andre301267/git/BD/Comexstat/IMP_COMPLETA_MUN.zip'
diretorio_destino = '/home/andre301267/git/BD/Comexstat/extracted_files/'

# Crie o diretório de destino, se não existir
os.makedirs(diretorio_destino, exist_ok=True)

# Abra o arquivo ZIP e extraia para o diretório de destino
with zipfile.ZipFile(caminho_arquivo_zip, 'r') as arquivo_zip:
    arquivo_zip.extractall(diretorio_destino)

print(f'Arquivos extraídos para: {diretorio_destino}')

Arquivos extraídos para: /home/andre301267/git/BD/Comexstat/extracted_files/


Lê in chunk e Imprime as primeiras 5 linhas do primeiro chunk, para verificar a estrutura do arquivo

In [6]:
reader=pd.read_csv('~/git/BD/Comexstat/extracted_files/IMP_COMPLETA_MUN.csv', chunksize=5, encoding='Latin-1',sep=';')

print(next(reader))

   CO_ANO  CO_MES   SH4  CO_PAIS SG_UF_MUN   CO_MUN  KG_LIQUIDO  VL_FOB
0    1997       6  9032      399        SP  3450308        1460  179845
1    1997       4  8421      249        SP  3424402         289   18950
2    1997      12  7616       23        MG  3106200        1312   20534
3    1997      12  8483      249        SC  4209102        4251  120559
4    1997       6  4104       63        RS  4307609       10828  247316


Nessa tabela acima, o produto é identificado na coluna SH4. Para interpretar esse código numérico e filtrar somente os fertilizantes, será necessário funidir a base de dados com a tabela auxiliar "NCM_SH.csv ", fornecida pelo site Comexstat.  

Vamos então abrir e visualizar a tabela "NCM_SH.csv":

In [7]:
ncm_sh=pd.read_csv('~/git/BD/Comexstat/NCM_SH.csv',encoding='Latin-1',sep=';')
ncm_sh.head(1)

,CO_SH6,NO_SH6_POR,NO_SH6_ESP,NO_SH6_ING,CO_SH4,NO_SH4_POR,NO_SH4_ESP,NO_SH4_ING,CO_SH2,NO_SH2_POR,NO_SH2_ESP,NO_SH2_ING,CO_NCM_SECROM,NO_SEC_POR,NO_SEC_ESP,NO_SEC_ING
0,10110,"Animais vivos das espécies cavalar, asinina e ...","Caballos y asnos, reproductores de raza pura",Pure-bred breeding horses and asses,101,"Cavalos, asininos e muares, vivos","Caballos, asnos, mulos y burdéganos, vivos","Live horses, asses, mules and hinnies",1,Animais vivos,Animales vivos,Live animals,I,Animais vivos e produtos do reino animal,Animales vivos y productos del reino animal,Lives animals; Animal products


In [8]:
ncm_sh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6609 entries, 0 to 6608
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   CO_SH6         6609 non-null   int64 
 1   NO_SH6_POR     6609 non-null   object
 2   NO_SH6_ESP     6609 non-null   object
 3   NO_SH6_ING     6609 non-null   object
 4   CO_SH4         6609 non-null   int64 
 5   NO_SH4_POR     6609 non-null   object
 6   NO_SH4_ESP     6609 non-null   object
 7   NO_SH4_ING     6609 non-null   object
 8   CO_SH2         6609 non-null   int64 
 9   NO_SH2_POR     6609 non-null   object
 10  NO_SH2_ESP     6609 non-null   object
 11  NO_SH2_ING     6609 non-null   object
 12  CO_NCM_SECROM  6609 non-null   object
 13  NO_SEC_POR     6609 non-null   object
 14  NO_SEC_ESP     6609 non-null   object
 15  NO_SEC_ING     6609 non-null   object
dtypes: int64(3), object(13)
memory usage: 826.3+ KB


O arquivo original o detalhamento de produto é SH4. Tentaremos simplificar buscando localizar os fertilizantes em um grupamento de produto mais amplo, o SH2

In [9]:
ncm_sh=ncm_sh[ncm_sh.columns[[4,5,8,9]]].drop_duplicates()
ncm_sh.head()

,CO_SH4,NO_SH4_POR,CO_SH2,NO_SH2_POR
0,101,"Cavalos, asininos e muares, vivos",1,Animais vivos
8,102,Animais vivos da espécie bovina,1,Animais vivos
14,103,Animais vivos da espécie suína,1,Animais vivos
17,104,Animais vivos das espécies ovina e caprina,1,Animais vivos
19,105,"Galos, galinhas, patos, gansos, perus, peruas ...",1,Animais vivos


Entre a base e dados e a tabela auxiliar, podemos notar que a colunas referentes ao código SH4 tem nomes diferentes. A seguir será alterado um dos nomes para iguala-los entre as tabelas.

In [10]:
ncm_sh=ncm_sh.rename(columns={'CO_SH4':'SH4'})

Na coluna com o nome do código SH2 em português, "NO_SH2_POR":

In [11]:
sh2_desejado=ncm_sh[ncm_sh.NO_SH2_POR.str.contains('fertilizantes')].CO_SH2.unique()
sh2_desejado

array([31])

Como só foi localizado um único código SH2 contendo o termo 'fertilizantes', salvaremos ele em variável para na sequencia filtrarmos a base de dados.

In [12]:
sh2_desejado=sh2_desejado[0]
sh2_desejado

31

O arquivo da base de dados é muito grande para ser aberto de uma só vez. Vamos então ler o arquivo em chuncks, fundi-lo (merge) com a tabela auxiliar ncm_sh, e filtrar cada chunk que contiver na coluna "CO_SH2" o valor contido  na variável "sh2_desejado".

In [13]:
%%time
reader=pd.read_csv('~/git/BD/Comexstat/extracted_files/IMP_COMPLETA_MUN.csv', chunksize=100_000, encoding='Latin-1',sep=';')
i_f_m=pd.DataFrame()
for chunk in reader:
    chunk=chunk.merge(ncm_sh)
    chunk=chunk[chunk.CO_SH2==sh2_desejado]
    i_f_m=pd.concat([i_f_m,chunk])
i_f_m.head(1)

CPU times: user 30.3 s, sys: 2.23 s, total: 32.6 s
Wall time: 32.8 s


,CO_ANO,CO_MES,SH4,CO_PAIS,SG_UF_MUN,CO_MUN,KG_LIQUIDO,VL_FOB,NO_SH4_POR,CO_SH2,NO_SH2_POR
53145,1997,7,3105,249,SP,3431308,3653780,734410,"Adubos (fertilizantes) minerais ou químicos, c...",31,Adubos (fertilizantes)


In [14]:
i_f_m.shape

(118136, 11)

In [15]:
i_f_m.duplicated(subset=['CO_ANO','CO_MES','SH4','CO_MUN','CO_PAIS']).sum()

0

Continuando a filtragem, dexando dessa vez passar somente as linhas onde a coluna SG_UF_MUN for igual a 'MG'

In [16]:
i_f_m=i_f_m[i_f_m.SG_UF_MUN=='MG']

Processando a última filtragem, agora deixando passar somente as linhas que correspndem aos municípios da região de Pouso Alegre. Para isso será utilizada a tabela auxiliar contendo os códigos e os nomes dos municípios, UF_MUN.csv

In [17]:
uf_mun=pd.read_csv('~/git/BD/Comexstat/UF_MUN.csv',encoding='Latin-1',sep=';')

Será modificado o nome deu uma das colunas de forma a igualar com a base de dados

In [18]:
uf_mun=uf_mun.rename(columns={'CO_MUN_GEO':'CO_MUN'})

Crrega a tabela  com os municípios da região de Pouso Alegre (m_r_pa)

In [19]:
m_r_pa=pd.read_csv('~/git/comex_sh_mun/temp/m_r_pa.csv').rename(columns={'Município':'NO_MUN_MIN'})

Procede a filtragem através da fundição das tabelas

In [20]:
i_f_m.duplicated().sum()

0

In [21]:
i_f_m_r_pa=uf_mun.merge(m_r_pa).merge(i_f_m)

In [22]:
i_f_m_r_pa.duplicated().sum()

0

In [23]:
# Salva a tabela resultante
i_f_m_r_pa.to_csv('~/git/comex_sh_mun/temp/i_f_m_r_pa.csv',index=False)

In [24]:
i_f_m_r_pa.head(1)

,CO_MUN,NO_MUN,NO_MUN_MIN,SG_UF,CO_ANO,CO_MES,SH4,CO_PAIS,SG_UF_MUN,KG_LIQUIDO,VL_FOB,NO_SH4_POR,CO_SH2,NO_SH2_POR
0,3125101,EXTREMA,Extrema,MG,2003,9,3105,493,MG,1000,323,"Adubos (fertilizantes) minerais ou químicos, c...",31,Adubos (fertilizantes)


In [25]:
i_f_m_r_pa.duplicated().sum()

0

Por fim, apaga a pasta onde o arquivo original e descompactado foi salvo

In [26]:
import shutil

diretorio_destino = '/home/andre301267/git/BD/Comexstat/extracted_files/'

# Excluir o diretório e todo o seu conteúdo
shutil.rmtree(diretorio_destino)

print(f'Diretório {diretorio_destino} e seu conteúdo foram excluídos.')


Diretório /home/andre301267/git/BD/Comexstat/extracted_files/ e seu conteúdo foram excluídos.


# 3- Exportação de Café  

No capítulo anterior, a base de dados de exportação por Município, denominada EXP_COMPLETA_MUN.zip, e as tabelas complementares, foram baixadas e salvas localmente na pasta:  

* '~/git/BD/Comexstat/'

In [70]:
import zipfile
import io
import os
import pandas as pd

Descompacta o arquivo e salva localmente em pasta específica

In [71]:
import zipfile
import os

caminho_arquivo_zip = '/home/andre301267/git/BD/Comexstat/EXP_COMPLETA_MUN.zip'
diretorio_destino = '/home/andre301267/git/BD/Comexstat/extracted_files/'

# Crie o diretório de destino, se não existir
os.makedirs(diretorio_destino, exist_ok=True)

# Abra o arquivo ZIP e extraia para o diretório de destino
with zipfile.ZipFile(caminho_arquivo_zip, 'r') as arquivo_zip:
    arquivo_zip.extractall(diretorio_destino)

print(f'Arquivos extraídos para: {diretorio_destino}')

Arquivos extraídos para: /home/andre301267/git/BD/Comexstat/extracted_files/


Lê in chunk e Imprime as primeiras 5 linhas do primeiro chunk, para verificar a estrutura do arquivo

In [72]:
reader=pd.read_csv('~/git/BD/Comexstat/extracted_files/EXP_COMPLETA_MUN.csv', chunksize=5, encoding='Latin-1',sep=';')

print(next(reader))

   CO_ANO  CO_MES   SH4  CO_PAIS SG_UF_MUN   CO_MUN  KG_LIQUIDO   VL_FOB
0    1997      11  8708       65        PR  4125506          90     3523
1    1997      11  8708      149        SP  3448708         324     2502
2    1997      11  9403      275        SC  4215802      909015  2367314
3    1997      11  6104      586        SC  4208906         710    23344
4    1997      11  8419      196        SP  3403208       96300   382100


A tabela auxiliar contendo os códigos e os nomes dos municípios, UF_MUN.csv

In [73]:
uf_mun=pd.read_csv('~/git/BD/Comexstat/UF_MUN.csv',encoding='Latin-1',sep=';')
uf_mun.head(1)

,CO_MUN_GEO,NO_MUN,NO_MUN_MIN,SG_UF
0,1501758,BREJO GRANDE DO ARAGUAIA,Brejo Grande do Araguaia,PA


Será modificado o nome de uma das colunas de forma a igualar com a base de dados

In [74]:
uf_mun=uf_mun.rename(columns={'CO_MUN_GEO':'CO_MUN'})

Carrega a tabela (que foi salva no item 1) com os municípios da região de Pouso Alegre (m_r_pa), para ser usada na filtragem os registros de exportação.

In [75]:
m_r_pa=pd.read_csv('~/git/comex_sh_mun/temp/m_r_pa.csv').rename(columns={'Município':'NO_MUN_MIN'})

In [76]:
co_mun_r_pa=uf_mun.merge(m_r_pa)

In [77]:
len(co_mun_r_pa[['CO_MUN']])

84

In [78]:
len(co_mun_r_pa[['CO_MUN']].drop_duplicates())

84

In [79]:
%%time
reader=pd.read_csv('~/git/BD/Comexstat/extracted_files/EXP_COMPLETA_MUN.csv', chunksize=10_000, encoding='Latin-1',sep=';')
e_m_r_pa=pd.DataFrame()
for chunk in reader:
    # Filtra cada chunk , deixando passar somente os registros dos municípios da região de Pouso Alegre
    chunk=chunk.merge(co_mun_r_pa)
    e_m_r_pa=pd.concat([e_m_r_pa,chunk])
e_m_r_pa.head(1)

CPU times: user 23 s, sys: 135 ms, total: 23.2 s
Wall time: 23.3 s


,CO_ANO,CO_MES,SH4,CO_PAIS,SG_UF_MUN,CO_MUN,KG_LIQUIDO,VL_FOB,NO_MUN,NO_MUN_MIN,SG_UF
0,1997,11,6210,586,MG,3159605,243,5162,SANTA RITA DO SAPUCAI,Santa Rita do Sapucaí,MG


Agora identificaremos qual produto agrícola é o mais relevante para a região foco, para que ele seja utilizado como variável multipla na previsão dos fertilizantes.  

Antes, em nome da simplicidade, o plano é prever os produtos utilizando o código de categora mais abrangente disponível, que é o código SH2, que é obtido através da fusão da base de dados com a tabela auxiliar, NCM_SH.csv, fornecida pelo site Comexstat.

In [80]:
ncm_sh=pd.read_csv('~/git/BD/Comexstat/NCM_SH.csv',encoding='Latin-1',sep=';').rename(columns={'CO_SH4':'SH4'})
ncm_sh.head(1)

,CO_SH6,NO_SH6_POR,NO_SH6_ESP,NO_SH6_ING,SH4,NO_SH4_POR,NO_SH4_ESP,NO_SH4_ING,CO_SH2,NO_SH2_POR,NO_SH2_ESP,NO_SH2_ING,CO_NCM_SECROM,NO_SEC_POR,NO_SEC_ESP,NO_SEC_ING
0,10110,"Animais vivos das espécies cavalar, asinina e ...","Caballos y asnos, reproductores de raza pura",Pure-bred breeding horses and asses,101,"Cavalos, asininos e muares, vivos","Caballos, asnos, mulos y burdéganos, vivos","Live horses, asses, mules and hinnies",1,Animais vivos,Animales vivos,Live animals,I,Animais vivos e produtos do reino animal,Animales vivos y productos del reino animal,Lives animals; Animal products


Descarta detalhamentos maiores (SH6) e condensa a tabela. Sem esse procedimento, nas operações merge a frente, duplicatas seriam criadas.

In [81]:
ncm_sh=ncm_sh[['SH4', 'NO_SH4_POR', 'CO_SH2', 'NO_SH2_POR']].drop_duplicates()

In [82]:
len(e_m_r_pa)

138345

In [83]:
e_m_r_pa=e_m_r_pa.merge(ncm_sh)

In [84]:
len(e_m_r_pa)

138345

In [85]:
e_m_r_pa.groupby(by=['CO_SH2','NO_SH2_POR']).VL_FOB.sum().sort_values(ascending=False).reset_index().head()

,CO_SH2,NO_SH2_POR,VL_FOB
0,9,"Café, chá, mate e especiarias",5520896891
1,84,"Reatores nucleares, caldeiras, máquinas, apare...",1876247134
2,85,"Máquinas, aparelhos e materiais elétricos, e s...",1321930431
3,26,"Minerios, escórias e cinzas",1051069830
4,28,Produtos químicos inorgânicos; compostos inorg...,913656789


In [86]:
e_c_m_r_pa=e_m_r_pa[e_m_r_pa.CO_SH2==9]

In [44]:
e_c_m_r_pa.head(1)

,CO_ANO,CO_MES,SH4,CO_PAIS,SG_UF_MUN,CO_MUN,KG_LIQUIDO,VL_FOB,NO_MUN,NO_MUN_MIN,SG_UF,NO_SH4_POR,CO_SH2,NO_SH2_POR
18709,1997,3,901,764,MG,3102605,150000,514554,ANDRADAS,Andradas,MG,"Café, mesmo torrado ou descafeinado; cascas e ...",9,"Café, chá, mate e especiarias"


Enfim, salvamos o arquivo, que esta pronto para uso.

In [45]:
e_c_m_r_pa.to_csv('~/git/comex_sh_mun/temp/e_c_m_r_pa.csv',index=False)

Apaga arquivo zip expandido

In [46]:
import shutil
import os

# Diretório a ser apagado
directory_to_delete = '~/git/BD/Comexstat/extracted_files/'

# Expandir o caminho do usuário
directory_to_delete = os.path.expanduser(directory_to_delete)

# Verificar se o diretório existe antes de apagar
if os.path.exists(directory_to_delete):
    # Apagar o diretório e todo o seu conteúdo
    shutil.rmtree(directory_to_delete)
    print(f'Diretório {directory_to_delete} apagado com sucesso.')
else:
    print(f'O diretório {directory_to_delete} não existe.')

Diretório /home/andre301267/git/BD/Comexstat/extracted_files/ apagado com sucesso.


# 4- Volumes de Chuva  

No capítulo anterior, a base de dados sobre chuvas, que é representada por um arquivo zip por ano de coleta, foi baixadas e salvas localmente na pasta:  

* '~/git/BD/Inmet/'

Extrai um dos arquivos zip para reconhecer o que esta no inteior

Limpa diretorio de extração

Extrai todos os arquivos zip

Move os arquivos extraídos para a pasta raíz

Elimina as subpastas que ficaram vazias

Remove todos os arquivos descompatados que não se refiram ao estado de MG

Carrega arquivo contendo lista de municípios da região de Pouso Alegre

Remove os arquivos que não contiverem no nome um nome de um dos município da região de Pouso Alegre

Divide as tabelas extraídas em duas, concatenando essas partes em uma planilha cada. Uma parte contedo a identificação das estações meteorológicas, outra parte contendo os dados climatológicos.

In [31]:
import os
import pandas as pd

# Diretório dos arquivos
diretorio = '/home/andre301267/git/BD/Inmet/extracted_files/'

# Lista para armazenar os DataFrames
df_parte1 = pd.DataFrame()
df_parte2 = pd.DataFrame()

# Iterar sobre os arquivos no diretório
for arquivo in os.listdir(diretorio):
    if arquivo.endswith('.CSV'):  # Certifique-se de que estamos lidando apenas com arquivos de texto
        caminho_arquivo = os.path.join(diretorio, arquivo)

        # Ler as primeiras 8 linhas
        parte1 = pd.read_csv(caminho_arquivo, nrows=7,sep=';',encoding='Latin-1')
       # print(parte1)

        # Ler a partir da linha 9
        parte2 = pd.read_csv(caminho_arquivo, skiprows=8,sep=';',encoding='Latin-1').assign(Estação=parte1.iloc[1,1])

        # Adicionar DataFrames às listas
        df_parte1=pd.concat([df_parte1,parte1],axis=1)
        df_parte2=pd.concat([df_parte2,parte2])

# Formata arquivo contendo ID das estações
df_parte1=df_parte1.transpose().drop(columns=[5,6]).drop_duplicates(subset=[1]).drop_duplicates(subset=[2])\
.reset_index(drop=True)

df_parte1.columns=df_parte1.iloc[0]

df_parte1=df_parte1.iloc[1:]


# Exibir os DataFrames
print("DataFrame Parte 1:")
display(df_parte1)

print("\nDataFrame Parte 2:")
display(df_parte2)

DataFrame Parte 1:


,UF:,ESTACAO:,CODIGO (WMO):,LATITUDE:,LONGITUDE:
1,MG,CONCEICAO DAS ALAGOAS,A520,"-19,98586","-48,151574"
2,MG,PASSA QUATRO,A529,"-22,395799","-44,961909"
3,MG,CALDAS,A530,"-21,918066","-46,382996"



DataFrame Parte 2:


,Data,Hora UTC,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (Kj/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),...,UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",Unnamed: 19,Estação,DATA (YYYY-MM-DD),HORA (UTC),RADIACAO GLOBAL (KJ/m²)
0,2020/01/01,0000 UTC,0,945,945,"944,2",NaN,"23,8",20,"24,6",...,74.0,79.0,58.0,0,0,NaN,CONCEICAO DAS ALAGOAS,NaN,NaN,NaN
1,2020/01/01,0100 UTC,0,"945,9",946,945,NaN,"23,6","19,4","24,1",...,75.0,78.0,2.0,"4,9","1,4",NaN,CONCEICAO DAS ALAGOAS,NaN,NaN,NaN
2,2020/01/01,0200 UTC,0,"945,8","946,1","945,8",NaN,"23,1","19,7",24,...,75.0,81.0,47.0,"5,1","1,1",NaN,CONCEICAO DAS ALAGOAS,NaN,NaN,NaN
3,2020/01/01,0300 UTC,0,"945,8",946,"945,8",NaN,"23,1",20,"23,3",...,80.0,83.0,30.0,"3,7","2,9",NaN,CONCEICAO DAS ALAGOAS,NaN,NaN,NaN
4,2020/01/01,0400 UTC,0,"945,5","945,9","945,5",NaN,"23,2",20,"23,4",...,82.0,82.0,349.0,"4,4","2,5",NaN,CONCEICAO DAS ALAGOAS,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2019/12/31,1900 UTC,0,891,"891,3",891,NaN,"26,6",NaN,"27,8",...,NaN,NaN,325.0,"5,7",",5",NaN,CALDAS,NaN,NaN,"1929,6"
8756,2019/12/31,2000 UTC,0,"890,7",891,"890,5",NaN,"24,4",NaN,"26,6",...,NaN,NaN,91.0,"4,3",2,NaN,CALDAS,NaN,NaN,"644,2"
8757,2019/12/31,2100 UTC,0,"890,8","890,9","890,6",NaN,"22,6",NaN,"24,7",...,NaN,NaN,105.0,"4,2",",1",NaN,CALDAS,NaN,NaN,295
8758,2019/12/31,2200 UTC,0,"890,8",891,"890,8",NaN,"21,1",NaN,"22,6",...,NaN,NaN,144.0,"1,2",0,NaN,CALDAS,NaN,NaN,"77,3"


Remove o diretório de extração

Salva arquivos

In [32]:
df_parte1.to_csv('~/git/comex_sh_mun/temp/estações_meteo.csv',index=False)
df_parte2.to_csv('~/git/comex_sh_mun/temp/chuvas_m_r_pa.csv',index=False)

In [33]:
df_parte2['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'].sort_values().unique()

array([',2', ',4', ',6', ',8', '-9999', '0', '1', '1,2', '1,4', '1,6',
       '1,8', '10', '10,2', '10,4', '10,6', '10,8', '11', '11,2', '11,4',
       '11,6', '11,8', '12', '12,2', '12,4', '12,6', '12,8', '13', '13,2',
       '13,4', '13,6', '13,8', '14', '14,2', '14,4', '14,6', '14,8', '15',
       '15,2', '15,4', '15,6', '15,8', '16', '16,2', '16,4', '16,6',
       '16,8', '17', '17,2', '17,4', '17,6', '17,8', '18', '18,2', '18,4',
       '18,6', '18,8', '19', '19,2', '19,4', '19,6', '19,8', '2', '2,2',
       '2,4', '2,6', '2,8', '20', '20,2', '20,4', '20,6', '20,8', '21',
       '21,2', '21,4', '21,6', '21,8', '22', '22,2', '22,4', '22,6',
       '22,8', '23', '23,2', '23,4', '23,6', '23,8', '24', '24,2', '24,4',
       '24,6', '24,8', '25', '25,2', '25,4', '25,6', '25,8', '26', '26,4',
       '26,6', '26,8', '27,2', '27,4', '27,6', '27,8', '28', '28,2',
       '28,4', '28,6', '28,8', '29', '29,2', '29,6', '3', '3,2', '3,4',
       '3,6', '3,8', '30', '30,2', '30,4', '30,6', '31,4